# 19. RNN with Hamlet

In [1]:
import torch
import torch.nn as nn

import nltk

import random
import numpy as np

## 19.1 Prepare Data

In [2]:
nltk.download("gutenberg")

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\slcf\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [32]:
raw = nltk.corpus.gutenberg.raw("shakespeare-hamlet.txt")
print(raw[:600])

[The Tragedie of Hamlet by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Barnardo and Francisco two Centinels.

  Barnardo. Who's there?
  Fran. Nay answer me: Stand & vnfold
your selfe

   Bar. Long liue the King

   Fran. Barnardo?
  Bar. He

   Fran. You come most carefully vpon your houre

   Bar. 'Tis now strook twelue, get thee to bed Francisco

   Fran. For this releefe much thankes: 'Tis bitter cold,
And I am sicke at heart

   Barn. Haue you had quiet Guard?
  Fran. Not a Mouse stirring

   Barn. Well, goodnight. If you do meet Horatio and
Marcellus, the Riuals of my 


## 19.2 Char to Dic

In [4]:
char2index = {}
index2char = []

for char in raw :
    if char not in char2index.keys() :
        char2index[char] = len(char2index)
        index2char.append(char)

In [5]:
char2index

{'[': 0,
 'T': 1,
 'h': 2,
 'e': 3,
 ' ': 4,
 'r': 5,
 'a': 6,
 'g': 7,
 'd': 8,
 'i': 9,
 'o': 10,
 'f': 11,
 'H': 12,
 'm': 13,
 'l': 14,
 't': 15,
 'b': 16,
 'y': 17,
 'W': 18,
 'S': 19,
 'k': 20,
 's': 21,
 'p': 22,
 '1': 23,
 '5': 24,
 '9': 25,
 ']': 26,
 '\n': 27,
 'A': 28,
 'c': 29,
 'u': 30,
 'P': 31,
 '.': 32,
 'n': 33,
 'E': 34,
 'B': 35,
 'F': 36,
 'w': 37,
 'C': 38,
 "'": 39,
 '?': 40,
 'N': 41,
 ':': 42,
 '&': 43,
 'v': 44,
 'L': 45,
 'K': 46,
 'Y': 47,
 ',': 48,
 'I': 49,
 'q': 50,
 'G': 51,
 'M': 52,
 'R': 53,
 '-': 54,
 'D': 55,
 'O': 56,
 'x': 57,
 ';': 58,
 'Q': 59,
 'z': 60,
 '(': 61,
 ')': 62,
 'V': 63,
 '!': 64,
 'j': 65,
 'Z': 66}

In [6]:
dim = len(char2index)
print("Dimension of input & output :", dim)

Dimension of input & output : 67


In [7]:
char2vec = {}
eye = np.eye(len(char2index))

for item in char2index.keys() :
    char2vec[item] = eye[char2index[item],:]

In [8]:
char2vec['a']

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
data = np.array([char2vec[char] for char in raw])

## 19.3 Define Model

In [10]:
# https://pytorch.org/docs/stable/nn.html#torch.nn.RNN

# Parameters:	
# input_size – The number of expected features in the input x
# hidden_size – The number of features in the hidden state h
# bias – If False, then the layer does not use bias weights b_ih and b_hh

In [11]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.hidden = torch.zeros(1, 1, hidden_size).cuda()
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out, hidden = self.rnn(x, self.hidden)
        self.hidden = hidden
        out = self.fc(out)
        return out

    def init_hidden(self):
        self.hidden = torch.zeros(1, 1, self.hidden_size).cuda()

In [12]:
model = RNN(dim, 250, dim).cuda()

## 19.4 Train Model

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [14]:
seq_len = 50
num_epochs = 3

In [15]:
for epoch in range(num_epochs):
    
    sp = list(range(0, len(data) - 2 * seq_len, seq_len))
    sp = np.add(sp, random.randint(0, seq_len))
    random.shuffle(sp)
    
    for i in range(len(sp)) :
    
        model.init_hidden()
        cost = 0
            
        for pos in range(sp[i], sp[i] + seq_len):
            
            X = torch.from_numpy(data[pos]).type(torch.FloatTensor)
            X = X.reshape(1, 1, dim).cuda()
            y = torch.from_numpy(data[pos+1]).cuda()
            
            _, y = y.max(dim=0)
            
            pre = model(X)
            cost += loss(pre.reshape(1, dim), y.reshape(1).cuda())

        optimizer.zero_grad()
        cost.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        
        optimizer.step()

        if (i + 1) % 1000 == 0 :
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                     %(epoch+1, num_epochs, i + 1, len(sp), cost.item()))

Epoch [1/3], Iter [100/3256] Loss: 174.2100
Epoch [1/3], Iter [200/3256] Loss: 145.6897
Epoch [1/3], Iter [300/3256] Loss: 131.9865
Epoch [1/3], Iter [400/3256] Loss: 142.8333
Epoch [1/3], Iter [500/3256] Loss: 123.4497
Epoch [1/3], Iter [600/3256] Loss: 127.7180
Epoch [1/3], Iter [700/3256] Loss: 136.6803
Epoch [1/3], Iter [800/3256] Loss: 123.8845
Epoch [1/3], Iter [900/3256] Loss: 124.9588
Epoch [1/3], Iter [1000/3256] Loss: 113.5791
Epoch [1/3], Iter [1100/3256] Loss: 95.0254
Epoch [1/3], Iter [1200/3256] Loss: 128.6707
Epoch [1/3], Iter [1300/3256] Loss: 113.9096
Epoch [1/3], Iter [1400/3256] Loss: 97.9635
Epoch [1/3], Iter [1500/3256] Loss: 126.5029
Epoch [1/3], Iter [1600/3256] Loss: 115.5672
Epoch [1/3], Iter [1700/3256] Loss: 125.6660
Epoch [1/3], Iter [1800/3256] Loss: 125.8327
Epoch [1/3], Iter [1900/3256] Loss: 129.0406
Epoch [1/3], Iter [2000/3256] Loss: 111.0097
Epoch [1/3], Iter [2100/3256] Loss: 129.7700
Epoch [1/3], Iter [2200/3256] Loss: 88.9129
Epoch [1/3], Iter [230

## 19.5 Test Model

In [16]:
start = 't'
text = start

model.eval()
model.init_hidden()

X_test = torch.from_numpy(char2vec[start]).type(torch.FloatTensor)
X_test = X_test.reshape(1, 1, dim).cuda()
    
for i in range(500) :

    pre = model(X_test)
    pre = pre.reshape(-1)
    
    _, pre = torch.max(pre, 0)
    new_char = index2char[pre]
    text += new_char
    
    X_test = torch.from_numpy(char2vec[new_char]).type(torch.FloatTensor)
    X_test = X_test.reshape(1, 1, dim).cuda()
    
print("* Generated Text : ")
print(text)

* Generated Text : 
ther and the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the King the Ki


## 19.6 Test Model with Best-5 Sampling

In [17]:
def get_top_index(x, num) :
    
    top_index = np.argsort(x)[::-1][:num]
    top_prob = x[top_index]
    
    # Softmax
    top_prob = np.exp(top_prob)
    top_prob = top_prob / top_prob.sum()
    
    random_index = np.random.choice(top_index, 1, p = top_prob)[0]
    
    return random_index

In [18]:
start = ' '
text = start

model.eval()
model.init_hidden()

X_test = torch.from_numpy(char2vec[start]).type(torch.FloatTensor)
X_test = X_test.reshape(1, 1, dim).cuda()

for i in range(500) :

    pre = model(X_test)
    pre = pre.reshape(-1).cpu().data.numpy()
    
    pre = get_top_index(pre, 5)
    new_char = index2char[pre]
    text += new_char
    
    X_test = torch.from_numpy(char2vec[new_char]).type(torch.FloatTensor)
    X_test = X_test.reshape(1, 1, dim).cuda()
    
print("* Generated Text : ")
print(text)

* Generated Text : 
 mar herringes, it in his the mante serues on a wall not him shile, all

   King. Hellenes them a dise,
But which a daucter, and heme, and this bue mad,
And whit the Kisg

   King., Is offerion,
Than shing of as is
Ant be wesce the ban a dowhing of the the that?
  Cporn. If a deetiell

   Qu. I am the thou hall his araund, and thous

   Ham. No mo great the King,
But leuse ties,
And thores your sine seene,
That the Aspences

   Ham. No, tho benteere this to haul that?
  Oppar.

  Hor. Now leat it
